In [79]:
import numpy as np
import pandas as pd
import datetime
import time
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

%run ./../custom_functions/train_dev_test_split.ipynb
%run ./../custom_functions/get_future_predictions.ipynb

from sklearn.ensemble import RandomForestRegressor

In [107]:
marag_df = pd.read_csv('./../../../Databases/clean_data/marag_lags_data.csv')
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)

marag_daily = pd.read_csv('./../../../Databases/clean_data/marag_dailylags_data.csv')
marag_daily.set_index('Timestamp', inplace=True)
marag_daily.sort_index(inplace=True)

# For the time being we will not use the outliers column, so we will drop it
marag_df.drop(['outliers'], axis=1, inplace=True)

In [81]:
# Let's import the marag data with no lags for future comparative purposes
no_lags_marag = pd.read_csv('./../../../Databases/clean_data/marag_data.csv')
no_lags_marag.set_index('Timestamp', inplace=True)
no_lags_marag.sort_index(inplace=True)
no_lags_marag.drop(['outliers'], axis=1, inplace=True)

daily_nolags_marag = pd.read_csv('./../../../Databases/clean_data/marag_daily_data.csv')
daily_nolags_marag.set_index('Timestamp', inplace=True)
daily_nolags_marag.sort_index(inplace=True)

In [10]:
# Let's create a dataframe in which we'll append all the estimated predictions, and another
# containing the running times of each scenario.

rf_results_df = pd.DataFrame
rf_runtime_df = pd.DataFrame

rf_dailyresults_df = pd.DataFrame
rf_runtime_dailydf = pd.DataFrame

## Model tuning

In [6]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])
rftune_X_train = pd.concat([X_train, X_dev])
rftune_X_train.sort_index(inplace=True)
rftune_y_train = pd.concat([y_train, y_dev])
rftune_y_train.sort_index(inplace=True)

In [8]:
# RandomForest baseline

rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)
RF_baseline_y_hat = rf.predict(X_dev)

/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
baseline_RMSE = pd.DataFrame(pd.Series(np.sqrt(mean_squared_error(y_dev, RF_baseline_y_hat)), name = 'RMSE'))
baseline_RMSE

,RMSE
0,33.85374


In [13]:
n_estimators = [100, 200, 400, 600]
max_depth = [10, 30, 60, 100]
best_RMSE = []

start_rf_tune = time.time()

for ne in tqdm(n_estimators):
    for md in tqdm(max_depth):
        rf = RandomForestRegressor(n_estimators = ne, max_depth = md, n_jobs = -1, random_state = 42)
        rf.fit(X_train, y_train)
        y_dev_hat = rf.predict(X_dev)
        RMSE_rfparams = sqrt(mean_squared_error(y_dev, y_dev_hat))
        
        if pd.DataFrame(best_RMSE).empty:
            best_RMSE = [ne, md, RMSE_rfparams]
        else:
            if RMSE_rfparams < best_RMSE[2]:
                best_RMSE = [ne, md, RMSE_rfparams]
        
end_rf_tune = time.time()
rf_tune_runtime = end_rf_tune - start_rf_tune



  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [01:25<04:17, 85.93s/it]


 50%|█████     | 2/4 [03:34<03:17, 98.70s/it]


 75%|███████▌  | 3/4 [05:45<01:48, 108.45s/it]


100%|██████████| 4/4 [07:58<00:00, 115.83s/it]

 25%|██▌       | 1/4 [07:58<23:56, 478.67s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [02:45<08:16, 165.62s/it]


 50%|█████     | 2/4 [07:01<06:25, 192.77s/it]


 75%|███████▌  | 3/4 [11:17<03:31, 211.67s/it]


100%|██████████| 4/4 [15:35<00:00, 225.67s/it]

 50%|█████     | 2/4 [23:34<20:31, 615.83s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [05:20<16:00, 320.09s/it]


 50%|█████     | 2/4 [13:52<12:35, 377.70s/it]


 75%|███████▌  | 3/4 [22:35<07:01, 421.41s/it]


100%|██████████| 4/4 [31:05<00:00, 447.97s/it]

 75%|███████▌  | 3/4 [54:40<16:30, 990.75s/it]


  0%|          | 0/4 [00:00<?, ?it/s]


 25%|██▌       | 1/4 [07:57<23:52, 477.61s/it]


 50%|█████     | 2/

In [14]:
print(best_RMSE, rf_tune_runtime)

[400, 30, 32.32575519464866] 6071.081792593002


The above tuning method is a simplification of the one intended to be used initially. The combination of hyperparameters implied the running of more than 4000 iterations and it was not a feasible achievement for the scope of this project (some of those iterations took up to 95 minutes to run).

## Original data (48 obsv. / day)

### All columns, known future

Future information will be used as the predictions will be forecast using lags of future observations.

In [11]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

rf_X_train = pd.concat([X_train, X_dev])
rf_X_train.sort_index(inplace=True)
rf_y_train = pd.concat([y_train, y_dev])
rf_y_train.sort_index(inplace=True)

In [12]:
start_PKF_allcols = time.time()

rf_PKF = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, verbose=10, random_state=42)
rf_PKF.fit(rf_X_train, rf_y_train)
rf_y_hat = rf_PKF.predict(X_test)
rf_y_hat = pd.Series(rf_y_hat, index=y_test.index, name='TotEntr_PKF_allcols')

end_PKF_allcols = time.time()
rf_runtime_PKF_allcols = end_PKF_allcols - start_PKF_allcols
rf_runtime_PKF_allcols = pd.DataFrame(pd.Series(rf_runtime_PKF_allcols, name = 'PKF_allcols_rf_runtime'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400

building tree 12 of 400
building tree 13 of 400
building tree 14 of 400


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.8s


building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    8.5s


...

[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  4.5min


building tree 354 of 400
building tree 355 of 400
building tree 356 of 400
building tree 357 of 400
building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400


[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:  4.8min


building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  5.2min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 176 tasks    

In [25]:
rf_observed_values = pd.DataFrame(y_test)
rf_observed_values.columns = ['Observed_values']
rf_results_df = rf_observed_values.merge(pd.DataFrame(rf_y_hat), how='left', on='Timestamp')
rf_results_df.head()

,Observed_values,TotEntr_PKF_allcols
Timestamp,,
2018-09-24 00:00:00,20,15.470000
2018-09-24 00:30:00,0,0.772719
2018-09-24 01:00:00,0,0.000000
2018-09-24 01:30:00,0,0.000000
2018-09-24 02:00:00,0,0.000000


In [18]:
rf_runtime_df = rf_runtime_PKF_allcols
rf_runtime_df.head()

,PKF_allcols_rf_runtime
0,312.858418


### Just lags, known future

Using just lags of future observations.

In [22]:
lags_PKF_df = marag_df.drop(['Temperature', 'Precipitation', 'Open/Closed', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, lags_X_test, y_train, y_dev, lags_y_test = train_dev_test_split(lags_PKF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_PKF_df['TotalEntries'])

rf_lags_X_train = pd.concat([X_train, X_dev])
rf_lags_X_train.sort_index(inplace=True)
rf_lags_y_train = pd.concat([y_train, y_dev])
rf_lags_y_train.sort_index(inplace=True)

In [28]:
start_PKF_lags = time.time()
rf_lags_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, verbose=10, random_state=42)
rf_lags_model.fit(rf_lags_X_train, rf_lags_y_train)
rf_lags_y_hat = rf_lags_model.predict(lags_X_test)
rf_lags_y_hat = pd.Series(rf_lags_y_hat, index=lags_y_test.index, name = 'TotEntr_PKF_lags')

end_PKF_lags = time.time()
rf_runtime_PKF_lags = end_PKF_lags - start_PKF_lags
rf_runtime_PKF_lags = pd.DataFrame(pd.Series(rf_runtime_PKF_lags, name = 'PKF_lags_rf_runtime'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400

building tree 13 of 400
building tree 14 of 400


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.9s


building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    8.3s


...

[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  4.5min


building tree 353 of 400
building tree 354 of 400
building tree 355 of 400
building tree 356 of 400
building tree 357 of 400
building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400


[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:  4.9min


building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  5.3min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 176 tasks    

In [29]:
rf_results_df = rf_results_df.merge(rf_lags_y_hat, on='Timestamp')
rf_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags
Timestamp,,,
2018-09-24 00:00:00,20,15.470000,15.475000
2018-09-24 00:30:00,0,0.772719,0.703395
2018-09-24 01:00:00,0,0.000000,0.000000
2018-09-24 01:30:00,0,0.000000,0.000000
2018-09-24 02:00:00,0,0.000000,0.000000


In [30]:
rf_runtime_df = pd.concat([rf_runtime_df, rf_runtime_PKF_lags], axis = 1)
rf_runtime_df.head()

,PKF_allcols_rf_runtime,PKF_lags_rf_runtime
0,312.858418,316.53875


### All columns, unknown future

Creating lags on the fly from new predictions.

In [108]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

rf_X_train = pd.concat([X_train, X_dev])
rf_X_train.sort_index(inplace=True)
rf_y_train = pd.concat([y_train, y_dev])
rf_y_train.sort_index(inplace=True)

In [117]:
rf_PUF_all_model = RandomForestRegressor(n_estimators=400, max_depth=30, n_jobs=-1, random_state=42)
rf_PUF_all_model.fit(rf_X_train, rf_y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.9s


building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    8.3s


...

building tree 353 of 400
building tree 354 of 400
building tree 355 of 400
building tree 356 of 400
building tree 357 of 400
building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400


[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:  4.9min


building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  5.3min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
           oob_score=False, random_state=42, verbose=10, warm_start=False)

A per previous tests run, each prediction takes an average of 5.4 minutes to be calculated using the predefined custom function. That means that for all rows on X_test (7392) it would be needed a complete month running non-stop in order to estimate all the predictions.

Therefore, the predictions for the first 10 days (480 observations) will be computed.

In [122]:
start_PUF_allcols = time.time()

rf_all_df = get_future_preds(no_lags_marag, rf_X_train, X_test[:480], rf_PUF_all_model)

end_PUF_allcols = time.time()
rf_runtime_PUF_allcols = end_PUF_allcols - start_PUF_allcols
rf_runtime_PUF_allcols = pd.DataFrame(pd.Series(rf_runtime_PUF_allcols, name = 'PUF_allcols_lr_runtime_10d'))
rf_all_future_df.columns = ['TotEntr_PUF_allcols_10d']

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 197 tasks      | elaps

building tree 1 of 400building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400

building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.9s


building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    8.6s


...

building tree 353 of 400
building tree 354 of 400
building tree 355 of 400
building tree 356 of 400
building tree 357 of 400
building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400


[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:  5.1min


building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  5.6min finished

100%|██████████| 3/3 [16:26<00:00, 328.06s/it]

NameError: name 'rf_all_future_df' is not defined

In [ ]:
#rf_results_df = rf_results_df.merge(rf_all_future_df, on='Timestamp')
#rf_results_df.head()

In [ ]:
rf_runtime_df = pd.concat([rf_runtime_df, rf_runtime_PUF_allcols], axis = 1)
rf_runtime_df.head()

In [112]:
X_grow = pd.concat([X_grow, y_hat])

In [113]:
X_grow.tail()

,TotalEntries
Timestamp,
2018-09-23 22:00:00,68.00
2018-09-23 22:30:00,67.00
2018-09-23 23:00:00,65.00
2018-09-23 23:30:00,36.00
2018-09-24 00:00:00,15.47


In [114]:
X_grow.sort_index()

TypeError: Cannot compare type 'Timestamp' with type 'str'

In [106]:
X_grow.tail().index

Index(['2018-09-23 22:00:00', '2018-09-23 22:30:00', '2018-09-23 23:00:00',
       '2018-09-23 23:30:00',   2018-09-24 00:00:00],
      dtype='object', name='Timestamp')

In [84]:
X_grow = pd.DataFrame(no_lags_marag['TotalEntries'][:X_train[-1:].index.astype(str)[0]].astype('float64').sort_index())

In [85]:
y_hat_init = pd.DataFrame(model.predict(X_test[0:1]), index=X_test[0:1].index, columns=['TotalEntries'])

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 197 tasks      | elaps

In [86]:
y_hat = y_hat_init

In [91]:
y_grow_hat.head()

TypeError: head() missing 1 required positional argument: 'self'

In [87]:
pd.concat([y_grow_hat, y_hat])

TypeError: cannot concatenate object of type "<class 'type'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid